In [22]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [23]:
import numpy as np
from IPython.display import Image

In [24]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

In [25]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [26]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [27]:
# 검증 데이터 분리
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]
# train 50000, val 10000, test 10000

In [28]:
# 데이터 구조 변경
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/reshape_mnist.png", width=500, height=250)

In [29]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)
(10000, 784)


In [30]:
# 데이터 정규화
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [31]:
# one hot encoding으로 변환
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [32]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
# 이거 구현할거임
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

Drop out

In [34]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [35]:
# 아래 하든 레이어2(h2)에 드랍아웃을 적용합니다. keep_prob의 값은 모델을 학습 또는 테스트할 때 결정합니다.
def mlp(x):
    # hidden layer 1
    w1 = tf.Variable(tf.random_uniform([784, 256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer 2
    w2 = tf.Variable(tf.random_uniform([256, 128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    h2_drop = tf.nn.dropout(h2, keep_prob)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128, 10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(h2_drop, w3) + b3
    
    return logits

In [36]:
logits = mlp(x)

In [37]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(\
                                                                   logits=logits, labels=y))

In [38]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

Early Stopping

In [39]:
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 300
batch_size = 1000
iteration = len(x_train) // batch_size

# 5번 연속으로 검증 정확도가 최고정확도보다 낮을경우 조기 종료
earlystop_threshold = 5
earlystop_cnt = 0

In [40]:
"""
데이터를 모델에 입력시킬 때(feed), 드랍아웃이 있을 경우, 항상 keep_prob를 설정해주셔야합니다.
학습 시, 10%의 드랍아웃을 하기 위해, keep_prob를 0.9로 설정합니다.
테스트 시, 드랍 아웃을 사용하지 않을 것이므로, keep_prob를 1.0으로 설정합니다.
"""
with tf.Session() as sess:
    sess.run(init)
    prev_train_acc = .0
    max_val_acc = .0
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op],
                              feed_dict={x: x_train[start:end],
                                        y: y_train[start:end], keep_prob: 0.9})
            start += batch_size; end += batch_size
            # 학습 손실값 계산
            avg_loss += loss / iteration
            
        # 모델 검증
        preds = tf.nn.softmax(logits) # 소프트맥스 적용
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # 정확도 계산
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # 학습 정확도 계산
        cur_train_acc = accuracy.eval({x: x_train, y: y_train, keep_prob: 1.0})
        # 검증 정확도 계산
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        
        print(f'epoch: {epoch}, train acc: {cur_train_acc}, val acc: {cur_val_acc}')
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print('early stopped on',epoch)
                    break
                else:
                    print('overfitting warning:',earlystop_cnt)
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # 검증 정확도 가장 높은 모델 저장
            save_path = saver.save(sess, 'model/model.ckpt')
        prev_train_acc = cur_train_acc

epoch: 0, train acc: 0.10202000290155411, val acc: 0.10320000350475311
epoch: 1, train acc: 0.47411999106407166, val acc: 0.48910000920295715
epoch: 2, train acc: 0.5546000003814697, val acc: 0.5756000280380249
epoch: 3, train acc: 0.6157199740409851, val acc: 0.6294999718666077
epoch: 4, train acc: 0.6685400009155273, val acc: 0.6858999729156494
epoch: 5, train acc: 0.7100800275802612, val acc: 0.7269999980926514
epoch: 6, train acc: 0.7397199869155884, val acc: 0.7565000057220459
epoch: 7, train acc: 0.7646999955177307, val acc: 0.7800999879837036
epoch: 8, train acc: 0.7850000262260437, val acc: 0.8001000285148621
epoch: 9, train acc: 0.8030800223350525, val acc: 0.8192999958992004
epoch: 10, train acc: 0.8195800185203552, val acc: 0.8352000117301941
epoch: 11, train acc: 0.8339599967002869, val acc: 0.8476999998092651
epoch: 12, train acc: 0.8446599841117859, val acc: 0.854200005531311
epoch: 13, train acc: 0.8552600145339966, val acc: 0.863099992275238
epoch: 14, train acc: 0.8641

epoch: 109, train acc: 0.9753599762916565, val acc: 0.9422000050544739
epoch: 110, train acc: 0.9778599739074707, val acc: 0.9434000253677368
epoch: 111, train acc: 0.9763200283050537, val acc: 0.9413999915122986
epoch: 112, train acc: 0.9762600064277649, val acc: 0.9444000124931335
epoch: 113, train acc: 0.9781399965286255, val acc: 0.9424999952316284
overfitting warning: 0
epoch: 114, train acc: 0.9777799844741821, val acc: 0.944100022315979
epoch: 115, train acc: 0.9780200123786926, val acc: 0.9430999755859375
overfitting warning: 0
epoch: 116, train acc: 0.9777600169181824, val acc: 0.9430999755859375
epoch: 117, train acc: 0.9760199785232544, val acc: 0.9408000111579895
epoch: 118, train acc: 0.9777399897575378, val acc: 0.9437999725341797
overfitting warning: 0
epoch: 119, train acc: 0.9768000245094299, val acc: 0.9437000155448914
epoch: 120, train acc: 0.975820004940033, val acc: 0.9429000020027161
epoch: 121, train acc: 0.9783599972724915, val acc: 0.9448999762535095
epoch: 122

epoch: 212, train acc: 0.9899399876594543, val acc: 0.9570000171661377
overfitting warning: 0
epoch: 213, train acc: 0.9922800064086914, val acc: 0.9577999711036682
epoch: 214, train acc: 0.9920799732208252, val acc: 0.957099974155426
overfitting warning: 0
epoch: 215, train acc: 0.9930400252342224, val acc: 0.9587000012397766
epoch: 216, train acc: 0.9932600259780884, val acc: 0.9584000110626221
overfitting warning: 0
epoch: 217, train acc: 0.9909800291061401, val acc: 0.9567999839782715
overfitting warning: 1
epoch: 218, train acc: 0.991100013256073, val acc: 0.9578999876976013
overfitting warning: 2
epoch: 219, train acc: 0.9895200133323669, val acc: 0.9562000036239624
epoch: 220, train acc: 0.9901999831199646, val acc: 0.955299973487854
overfitting warning: 0
epoch: 221, train acc: 0.9919800162315369, val acc: 0.9581999778747559
overfitting warning: 1
epoch: 222, train acc: 0.9921000003814697, val acc: 0.9591000080108643
epoch: 223, train acc: 0.9933800101280212, val acc: 0.9603000

In [41]:
# 테스트 시작
with tf.Session() as sess:
    # 검증 결과가 가장 높았던 모델 불러오기
    saver.restore(sess, 'model/model.ckpt')
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # 정확도 계산
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.9593
